# Section 5: deep reinforcement learning with the openai gym

Reinforcement learning is the process of teaching an autonomous agent how to behave in an environment. Rather than telling the agent what action to execute at each moment you let the agent decide for himself what to do, and you give it rewards based on how it performs. 

Recently there is a lot of attention on reinforcement learning. One reason is that the company [OpenAI](https://en.wikipedia.org/wiki/OpenAI) released a set of environments you can use to train your agent. The environment we will be using is the OpenAI gym. There are many tasks in this package which have defined how much reward you get for each action. Take a look at the [environments that are included with the OpenAI gym](https://gym.openai.com/envs/). As you can see there are classic control problems, but OpenAI also has a lot of Atari games you can try to master!

# Video 1: getting started with the OpenAI gym
### Installing the OpenAI gym
There are multiple ways you can install the OpenAI gym environment: 

##### In our Docker image
It is already installed in our Docker image. This means you don't have to do anything to include it, and you can start playing immediately. The downside of this is that there is no screen attached to our Docker image, which gives a bit of hassle inline. 

##### On your own pc. 
If you do want to see a screen with your game you would have to install everything locally. This means you have your own version of Python, TensorFlow, Matplotlib, etc. installed. If this is all fine you can install the OpenAI gym with these commands: 
> `git clone https://github.com/openai/gym.git`

> `cd gym`

> `pip install -e .`

For more information: see [this site](https://github.com/openai/gym#installing-everything). 

### Starting an environment
Let's start by loading some libraries we want to use during this section. They are the normal ones you expect by now (TensorFlow, NumPy, etc...). We also load gym!

In [3]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random

from ipywidgets import widgets
from IPython.display import display

import gym

## Set logging dir for Tensorboard
logging_dir_n = 141


To load an environment you call the `gym.make` function and pass the name of the environment you want to load. After you reset you get the observation. If you installed gym on your local pc you can get a screen with the state of your environment using the command `env.render()`. As I will be doing everything from this Docker image we render the frame as RGB array and display it with Matplotlib...

In [5]:
env = gym.make('CartPole-v0')
observation = env.reset()
firstframe = env.render(mode = 'rgb_array')
plt.imshow(firstframe)

NameError: name 'base' is not defined

### Performing (random) actions
With the function `env.action_space.sample()` you get a random action that's possible for this action space. Please take some time to figure out what kind of commands are possible, and what env.step does. 

In the following code we execute a few actions, plot the frames, and print what the environment tells us about the current state of the environment...

In [9]:
observation = env.reset()
for _ in range(30):
    frame = env.render(mode = 'rgb_array')
    plt.imshow(frame)
    plt.show()
    observation, reward, done, info = env.step(env.action_space.sample()) 
    print("Observation: %s reward: %d done: %d info: %s" % (str(observation), reward, done, info))

You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.


Observation: [-0.0263599   0.23144936  0.01105679 -0.28842569] reward: 1 done: 0 info: {}
Observation: [-0.02173092  0.03617149  0.00528828  0.00772384] reward: 1 done: 0 info: {}
Observation: [-0.02100749 -0.1590259   0.00544275  0.30207058] reward: 1 done: 0 info: {}
Observation: [-0.024188    0.03601806  0.01148416  0.01110913] reward: 1 done: 0 info: {}
Observation: [-0.02346764 -0.15926669  0.01170635  0.30739321] reward: 1 done: 0 info: {}
Observation: [-0.02665298 -0.35455348  0.01785421  0.60374491] reward: 1 done: 0 info: {}
Observation: [-0.03374405 -0.54992052  0.02992911  0.90199763] reward: 1 done: 0 info: {}
Observation: [-0.04474246 -0.74543488  0.04796906  1.20393567] reward: 1 done: 0 info: {}
Observation: [-0.05965115 -0.94114299  0.07204778  1.51125765] reward: 1 done: 0 info: {}
Observation: [-0.07847401 -0.74696397  0.10227293  1.24190848] reward: 1 done: 0 info: {}
Observation: [-0.09341329 -0.94323914  0.1271111   1.56479869] reward: 1 done: 0 info: {}
Observatio

### Input and output 

Let's take a look at what we can use to train an autonomous agent: 

##### Observation
The observation consists of the angle of the pole, turn velocity of the pole, the location of the base, and the movespeed of the base. Other environments have different inputs. Some environments (the Atari games) simply give you an image, other environments give a representation of the state. 

##### Reward
Reward is the reward our environment gives us each frame. For cartpole the reward is 1.0 while we managed to keep the stick in less than a small angle. After we reached this angle the reward will change to 0.0 while we did not reset the environment. 

Other environments give different rewards. Mountaincar for example gives a reward of 0.0 while we did not reach the top, and 1.0 once we reached it. Whenever you try a new environment make sure you know when your algorithm receives a reward as this is vital for learning something. 

##### Done
If this boolean is True it means you probably want to reset your network. In the case of cartpole it means the cart can't get anymore points: either by:
- failing to keep the pole at less than a certain angle
- failing to keep the base at less than a certain distance
- succeeding at this for more than 200 frames!

Other environments end when your lives are gone, others end after a certain amount of iterations.

#### Actions
What actions you can perform depend on the environment of you agent. In this environment you can only perform two actions: apply a force to the left of the base, or to the right of the base. In other environments, like the Atari environment, you can press several buttons!

### Playing yourself
Now you are able to play a pretty boring game with a text-interface...

In [ ]:
# observation = env.reset()
# done = False
# while not done:
#     frame = env.render(mode = 'rgb_array')
#     plt.imshow(frame)
#     plt.show()
    
#     useraction = -1
#     while useraction < 0:
#         try:
#             useraction = int(input("Left (0) or right (1)?"))
#         except:
#             pass
        
        
#     observation, reward, done, info = env.step(useraction) 
# print("You are dead..")

## Video 2: Random search
One way you could approach this problem is with a very simple single-layer neural network that takes the input and has weights to two actions. You can imagine that this neural network makes sure the left action is chosen if the stick is too far to the left, and the right action is chosen if the stick is too far to the right. 

Let's start with this simple single-layer neural network. As code I took the example from [this website](http://kvfrans.com/simple-algoritms-for-solving-cartpole/). If you like and have time I would definitely explore all samples of that blogpost as it gives a lot of insight in reinforcement learning and the cartpole problem!

For now it is enough to say that we can construct a single-layer neural network by having a matrix multiplication of the state with a 4x1 matrix (if the result is smaller than 0 we push left!). To get this matrix we are going to do generate A LOT of random matrices and see which ones perform best. This technique, random search, is described on the blog I linked to earlier. You can copy-paste the code from the blog, and change it however you like it. 


In [22]:
def run_episode(env, parameters):  
    observation = env.reset()
    totalreward = 0
    for _ in range(200):
        action = 0 if np.matmul(parameters,observation) < 0 else 1
        observation, reward, done, info = env.step(action)
        totalreward += reward
        if done:
            break
    return totalreward


bestparams = None  
bestreward = 0  
for iteration in range(10000):  
    parameters = np.random.rand(4) * 2 - 1
    reward = run_episode(env,parameters)
    if reward > bestreward:
        bestreward = reward
        bestparams = parameters
        # considered solved if the agent lasts 200 timesteps
        if reward == 200:
            print("WOW! FOUND A SOLUTION TO THIS PROBLEM USING RANDOM SEARCH AT ITERATION %d!" % (iteration))
            break
    print("Solution not yet found")

Solution not yet found
Solution not yet found
WOW! FOUND A SOLUTION TO THIS PROBLEM USING RANDOM SEARCH AT ITERATION 2!


Try to understand what is happening in the run_episode function, and what is happening in the search function above. After we did this it's time to visualise the best, winning, parameters: 


In [24]:
observation = env.reset()
frames = list()
for _ in range(200):
    action = 0 if np.matmul(bestparams,observation) < 0 else 1
    observation, reward, done, info = env.step(action)
    
    frame = env.render(mode = 'rgb_array')
    frames.append(frame)
    
    if done:
        break

NameError: name 'base' is not defined

Now that we have a list with frames we can render them as a GIF. I use the following function to construct the gif and show it inline in our Jupyter notebook: 

In [ ]:
from matplotlib import animation
from JSAnimation.IPython_display import display_animation

def display_frames_as_gif(frames, filename_gif = None):
    """
    Displays a list of frames as a gif, with controls
    """
    plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval=50)
    if filename_gif: 
        anim.save(filename_gif, writer = 'imagemagick', fps=20)
    display(display_animation(anim, default_mode='loop'))

    
display_frames_as_gif(frames)


## Video 3: Reinforcement learning explained
Now that we are able to find the best parameters for this simple problem we can also realize that this approach is not going to work on a more interesting problem. That's why we are going to learn what [reinforcement learning](https://en.wikipedia.org/wiki/Reinforcement_learning), and specifically [q-learning](https://en.wikipedia.org/wiki/Q-learning), can do!

Imagine that you are playing a game of super mario brothers. This game gives you certain rewards when you: 
- Grab a coin
- Kill a goomba
- Finish a level

It also gives you penalties whener you fall off a cliff or walk against an enemy... 

The first time you played the game you simply pressed random buttons, and learned after you got penalties and rewards after you accidentally performed something good. 

The idea of reinforcement learning is letting an agent perform random actions and learn from them. To do this we construct a function that can determine the quality of a state-action pair. 

Let's say you are in a certain state. For each action you can estimate the reward you expect to get performing that action. In addition to this estimated reward we want to be able to obtain a high reward in the long term. Jumping off a cliff to grab that one extra coin is not worth it if you can walk a bit further and gain a lot of coins without dying!

The function that estimates the (future) reward for an action and a state is indicated with Q(s,a). We can represent this with a neural network that takes a state as input and estimates the (future) reward of that state for each action. In case of the cartpole environment we take 4 values as input, and estimate two values (two possible actions) as output...

We update this function by looking at what state we reached after performing a certain action. As we now have a new state we assume our function is correct, and the future reward can be estimated with Q(next_s, a), taking the value of a which leads to the highest reward. We multiply the next state with a discount-constant gamma. A high gamma (close to 1) values long-term rewards, while a low gamma (closer to 0) only values short-term rewards...

The mathematical expression we use to update our q-function is: 

![reward fucntion](http://www.pinchofintelligence.com/wp-content/uploads/2017/07/rewardfunction.png)

### Implement it in TensorFlow
There are multiple ways to construct the reward-estimation function. In our case we are going to build a neural network with TensorFlow that estimates the expected (future)reward. 

To do this we need three input placeholders: the state, action our network took, and the reward it eventually got. This reward it eventually got is what we get from the same network!

In [ ]:
tf.reset_default_graph()
networkstate = tf.placeholder(tf.float32, [None, 4], name="input")
networkaction = tf.placeholder(tf.int32, [None], name="actioninput")
networkreward = tf.placeholder(tf.float32,[None], name="groundtruth_reward")

layer1 = tf.layers.dense(networkstate, 16, activation=tf.nn.relu, name="dense1")
layer2 = tf.layers.dense(layer1, 32, activation=tf.nn.relu, name="dense2")
layer3 = tf.layers.dense(layer2, 8, activation=tf.nn.relu, name="dense3")
predictedreward = tf.layers.dense(layer3, 2, name="dense4")


In [ ]:
with tf.name_scope("lossdeterminer"):
    action_onehot = tf.one_hot(networkaction, 2, name="actiononehot")
    qreward = tf.reduce_sum(tf.multiply(predictedreward, action_onehot), reduction_indices = 1)
    loss = tf.reduce_mean(tf.square(networkreward - qreward))

tf.summary.scalar('loss', loss)
tf.summary.scalar('mean_max_reward', tf.reduce_max(predictedreward))

#optimizer = tf.train.AdamOptimizer().minimize(loss)
optimizer = tf.train.RMSPropOptimizer(0.01).minimize(loss)
merged_summary = tf.summary.merge_all()


In [ ]:
sess = tf.InteractiveSession()
summary_writer = tf.summary.FileWriter('graphs/dqn/normal/'+str(logging_dir_n),sess.graph)
logging_dir_n +=1
sess.run(tf.global_variables_initializer())


### Setting hyperparameters

In [ ]:
BATCH_SIZE = 128
GAMMA = 0.9
MAX_LEN_REPLAY_MEMORY = 10000 # replay memory of one million most recent frames.
FRAMES_TO_PLAY = 50001
MIN_FRAMES_FOR_LEARNING = 1000

### Training

In [ ]:
def learn(replay_memory):
      ### Learn once we have enough frames to start learning
    if len(replay_memory) > MIN_FRAMES_FOR_LEARNING: 
        experiences = random.sample(replay_memory, BATCH_SIZE)
        totrain = [] # (state, action, delayed_reward)
        
        ### Calculate the predicted reward
        nextstates = [var[4] for var in experiences]
        pred_reward = sess.run(predictedreward, feed_dict={networkstate:nextstates})
        
        ### Set the "ground truth": the value our network has to predict:
        for index in range(BATCH_SIZE):
            state, action, reward, terminalstate, newstate = experiences[index]
            predicted_reward = max(pred_reward[index])
            
            if terminalstate:
                delayedreward = reward
            else:
                delayedreward = reward + GAMMA*predicted_reward
            totrain.append((state, action, delayedreward))
            
        ### Feed the train batch to the algorithm 
        states = [var[0] for var in totrain]
        actions = [var[1] for var in totrain]
        rewards = [var[2] for var in totrain]
        _, l, summary = sess.run([optimizer, loss, merged_summary], feed_dict={networkstate:states, networkaction: actions, networkreward: rewards})


        ### If our memory is too big: remove the first element
        if len(replay_memory) > MAX_LEN_REPLAY_MEMORY:
                replay_memory = replay_memory[1:]

        ### Show the progress 
        summary_writer.add_summary(summary, i_epoch)

  

### Running this in our environment

In [ ]:
replay_memory = [] # (state, action, reward, terminalstate, state_t+1)

for i_epoch in range(FRAMES_TO_PLAY):
    
    ### Select an action and perform this
    action = env.action_space.sample() 
    newobservation, reward, terminal, info = env.step(action)

    ### I prefer that my agent gets 0 reward if it dies
    if terminal: 
        reward = 0
        
    ### Add the observation to our replay memory
    replay_memory.append((observation, action, reward, terminal, newobservation))
    
    ### Reset the environment if the agent died
    if terminal: 
        newobservation = env.reset()
    observation = newobservation
    
    learn(replay_memory)
    

### Now test this

RUN pip install git+https://github.com/jakevdp/JSAnimation.git

Use imagemagick, 



In [ ]:
from matplotlib import animation
from JSAnimation.IPython_display import display_animation

def display_frames_as_gif(frames, filename_gif = None):
    """
    Displays a list of frames as a gif, with controls
    """
    plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval=50)
    if filename_gif: 
        anim.save(filename_gif, writer = 'imagemagick', fps=20)
    display(display_animation(anim, default_mode='loop'))


In [ ]:
observation = env.reset()
term = False
predicted_q = []
frames = []

### Play till we are dead
while not term:
    rgb_observation = env.render(mode = 'rgb_array')
    frames.append(rgb_observation)
    pred_q = sess.run(predictedreward, feed_dict={networkstate:[observation]})
    predicted_q.append(pred_q)
    action = np.argmax(pred_q)
    observation, _, term, _ = env.step(action)

### Plot the replay!
print("Frames: " + str(len(frames)))
display_frames_as_gif(frames,filename_gif='dqn_run.gif')

### Reinforcement learning tricks


In [ ]:
sess = tf.InteractiveSession()
summary_writer = tf.summary.FileWriter('graphs/dqn/normal/'+str(logging_dir_n),sess.graph)
logging_dir_n +=1
sess.run(tf.global_variables_initializer())

In [ ]:
replay_memory = [] # (state, action, reward, terminalstate, state_t+1)
epsilon = 1.0
decay_factor = 0.001
MIN_EPSILON = 0.1

observation = env.reset()

for i_epoch in range(FRAMES_TO_PLAY):
    
    ### Select action
    if random.random() < epsilon:
        action = env.action_space.sample() 
    else:
        pred_q = sess.run(predictedreward, feed_dict={networkstate:[observation]})
        action = np.argmax(pred_q)
    
    ### Anneal epsilon
    epsilon = max(epsilon-decay_factor, MIN_EPSILON)
    
    newobservation, reward, terminal, info = env.step(action)
    ### I prefer that my agent gets 0 reward if it dies
    if terminal: 
        reward = 0
        
    ### Add the observation to our replay memory
    replay_memory.append((observation, action, reward, terminal, newobservation))
    
    ### Reset the environment if the agent died
    if terminal: 
        newobservation = env.reset()
    observation = newobservation
    
    learn(replay_memory)

In [ ]:
observation = env.reset()
term = False
predicted_q = []
frames = []

### Play till we are dead
while not term:
    rgb_observation = env.render(mode = 'rgb_array')
    frames.append(rgb_observation)
    pred_q = sess.run(predictedreward, feed_dict={networkstate:[observation]})
    predicted_q.append(pred_q)
    action = np.argmax(pred_q)
    observation, _, term, _ = env.step(action)

### Plot the replay!
print("Frames: " + str(len(frames)))
display_frames_as_gif(frames,filename_gif='dqn_run.gif')

## Video 4: Remembering your actions: adding a replay memory


## Video 5: Introduction to the Atari environment


In [ ]:
env = gym.make("BreakoutDeterministic-v4")
observation = env.reset()
print(observation.shape)
plt.imshow(observation)

observation = env.reset()
for _ in range(10):
    action = env.action_space.sample()
    print(action)
    observation, reward, done, info = env.step(action) 
    plt.imshow(observation)
    plt.show()



In [ ]:
observation.shape

https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf

In [ ]:
env.action_space.n

In [ ]:
def huber_loss(x, delta=1.0):
    """Reference: https://en.wikipedia.org/wiki/Huber_loss"""
    return tf.where(
        tf.abs(x) < delta,
        tf.square(x) * 0.5,
        delta * (tf.abs(x) - 0.5 * delta)
    )

def scope_vars(scope, trainable_only=False):
    """
    Get variables inside a scope
    The scope can be specified as a string
    Parameters
    ----------
    scope: str or VariableScope
        scope in which the variables reside.
    trainable_only: bool
        whether or not to return only the variables that were marked as trainable.
    Returns
    -------
    vars: [tf.Variable]
        list of variables in `scope`.
    """
    return tf.get_collection(
        tf.GraphKeys.TRAINABLE_VARIABLES if trainable_only else tf.GraphKeys.GLOBAL_VARIABLES,
        scope=scope if isinstance(scope, str) else scope.name
    )

def scope_name():
    """Returns the name of current scope as a string, e.g. deepq/q_func"""
    return tf.get_variable_scope().name

def absolute_scope_name(relative_scope_name):
    """Appends parent scope name to `relative_scope_name`"""
    return scope_name() + "/" + relative_scope_name


def function(inputs, outputs, updates=None, givens=None):
    """Just like Theano function. Take a bunch of tensorflow placeholders and expressions
    computed based on those placeholders and produces f(inputs) -> outputs. Function f takes
    values to be fed to the input's placeholders and produces the values of the expressions
    in outputs.
    Input values can be passed in the same order as inputs or can be provided as kwargs based
    on placeholder name (passed to constructor or accessible via placeholder.op.name).
    Example:
        x = tf.placeholder(tf.int32, (), name="x")
        y = tf.placeholder(tf.int32, (), name="y")
        z = 3 * x + 2 * y
        lin = function([x, y], z, givens={y: 0})
        with single_threaded_session():
            initialize()
            assert lin(2) == 6
            assert lin(x=3) == 9
            assert lin(2, 2) == 10
            assert lin(x=2, y=3) == 12
    Parameters
    ----------
    inputs: [tf.placeholder or TfInput]
        list of input arguments
    outputs: [tf.Variable] or tf.Variable
        list of outputs or a single output to be returned from function. Returned
        value will also have the same shape.
    """
    if isinstance(outputs, list):
        return _Function(inputs, outputs, updates, givens=givens)
    elif isinstance(outputs, (dict, collections.OrderedDict)):
        f = _Function(inputs, outputs.values(), updates, givens=givens)
        return lambda *args, **kwargs: type(outputs)(zip(outputs.keys(), f(*args, **kwargs)))
    else:
        f = _Function(inputs, [outputs], updates, givens=givens)
        return lambda *args, **kwargs: f(*args, **kwargs)[0]

class _Function(object):
    def __init__(self, inputs, outputs, updates, givens, check_nan=False):
        for inpt in inputs:
            if not issubclass(type(inpt), TfInput):
                assert len(inpt.op.inputs) == 0, "inputs should all be placeholders of baselines.common.TfInput"
        self.inputs = inputs
        updates = updates or []
        self.update_group = tf.group(*updates)
        self.outputs_update = list(outputs) + [self.update_group]
        self.givens = {} if givens is None else givens
        self.check_nan = check_nan

    def _feed_input(self, feed_dict, inpt, value):
        if issubclass(type(inpt), TfInput):
            feed_dict.update(inpt.make_feed_dict(value))
        elif is_placeholder(inpt):
            feed_dict[inpt] = value

    def __call__(self, *args, **kwargs):
        assert len(args) <= len(self.inputs), "Too many arguments provided"
        feed_dict = {}
        # Update the args
        for inpt, value in zip(self.inputs, args):
            self._feed_input(feed_dict, inpt, value)
        # Update the kwargs
        kwargs_passed_inpt_names = set()
        for inpt in self.inputs[len(args):]:
            inpt_name = inpt.name.split(':')[0]
            inpt_name = inpt_name.split('/')[-1]
            assert inpt_name not in kwargs_passed_inpt_names, \
                "this function has two arguments with the same name \"{}\", so kwargs cannot be used.".format(inpt_name)
            if inpt_name in kwargs:
                kwargs_passed_inpt_names.add(inpt_name)
                self._feed_input(feed_dict, inpt, kwargs.pop(inpt_name))
            else:
                assert inpt in self.givens, "Missing argument " + inpt_name
        assert len(kwargs) == 0, "Function got extra arguments " + str(list(kwargs.keys()))
        # Update feed dict with givens.
        for inpt in self.givens:
            feed_dict[inpt] = feed_dict.get(inpt, self.givens[inpt])
        results = get_session().run(self.outputs_update, feed_dict=feed_dict)[:-1]
        if self.check_nan:
            if any(np.isnan(r).any() for r in results):
                raise RuntimeError("Nan detected")
        return results

def mem_friendly_function(nondata_inputs, data_inputs, outputs, batch_size):
    if isinstance(outputs, list):
        return _MemFriendlyFunction(nondata_inputs, data_inputs, outputs, batch_size)
    else:
        f = _MemFriendlyFunction(nondata_inputs, data_inputs, [outputs], batch_size)
        return lambda *inputs: f(*inputs)[0]

class _MemFriendlyFunction(object):
    def __init__(self, nondata_inputs, data_inputs, outputs, batch_size):
        self.nondata_inputs = nondata_inputs
        self.data_inputs = data_inputs
        self.outputs = list(outputs)
        self.batch_size = batch_size

    def __call__(self, *inputvals):
        assert len(inputvals) == len(self.nondata_inputs) + len(self.data_inputs)
        nondata_vals = inputvals[0:len(self.nondata_inputs)]
        data_vals = inputvals[len(self.nondata_inputs):]
        feed_dict = dict(zip(self.nondata_inputs, nondata_vals))
        n = data_vals[0].shape[0]
        for v in data_vals[1:]:
            assert v.shape[0] == n
        for i_start in range(0, n, self.batch_size):
            slice_vals = [v[i_start:builtins.min(i_start + self.batch_size, n)] for v in data_vals]
            for (var, val) in zip(self.data_inputs, slice_vals):
                feed_dict[var] = val
            results = tf.get_default_session().run(self.outputs, feed_dict=feed_dict)
            if i_start == 0:
                sum_results = results
            else:
                for i in range(len(results)):
                    sum_results[i] = sum_results[i] + results[i]
        for i in range(len(results)):
            sum_results[i] = sum_results[i] / n
        return sum_results


In [ ]:
GAMMA = 0.99

tf.reset_default_graph()
global_step = tf.Variable(0, trainable=False)

networkstate = tf.placeholder(tf.float32, [None, 84, 84, 4], name="input")
nextnetworkstate = tf.placeholder(tf.float32, [None, 84, 84, 4], name="inputnext")
networkaction = tf.placeholder(tf.int32, [None], name="actioninput")
networkreward = tf.placeholder(tf.float32, [None], name="actioninput")
done_mask_ph = tf.placeholder(tf.float32, [None], name="done")


inputimage = networkstate/255
nextnetworkstate = nextnetworkstate/255
# inputimage = tf.image.resize_images(inputimage, (84, 84))

#tf.summary.image("inputimage", inputimage)

def build_network(namescope, input_for_network):
    with tf.variable_scope(namescope):
        layer1 = tf.layers.conv2d(inputimage, 32, [8,8], activation=tf.nn.relu, strides=(4, 4), name="conv1", padding='same')
        layer2 = tf.layers.conv2d(layer1, 64, [4,4], strides=(2,2), activation=tf.nn.relu, name="conv2", padding='same')
        layer3 = tf.layers.conv2d(layer2, 64, [3,3], activation=tf.nn.relu, name="conv3", padding="same")
        flattened = tf.contrib.layers.flatten(layer3)

        flat2 = tf.layers.dense(flattened, 256, activation=tf.nn.relu, name="dense2")
        predictedreward = tf.layers.dense(flat2, env.action_space.n, name="dense3")
    return predictedreward

predicted_action = build_network("actionnetwork", inputimage)
#q_func_vars = scope_vars(absolute_scope_name("actionnetwork"))
q_func_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "actionnetwork")
assert len(q_func_vars) > 0

predicted_value = build_network("valuenetwork", nextnetworkstate)
#target_q_func_vars = scope_vars(absolute_scope_name("valuenetwork"))
target_q_func_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "valuenetwork")
assert len(target_q_func_vars) > 0

with tf.name_scope("lossdeterminer"):
    action_onehot = tf.one_hot(networkaction, env.action_space.n, name="actiononehot")
    predicted_qreward = tf.reduce_sum(tf.multiply(predicted_action, action_onehot), reduction_indices = 1)
    ## TODO: Do I want to set estimated reward to zero if the agent was done in this frame?
    estimated_reward = tf.reduce_max(predicted_value)

    q_tp1_best_masked = (1.0 - done_mask_ph) * estimated_reward

    q_t_selected_target = networkreward + GAMMA * q_tp1_best_masked
    td_error = predicted_qreward - tf.stop_gradient(q_t_selected_target)
    loss = huber_loss(td_error)

#     loss = tf.reduce_mean(tf.square(networkreward - qreward))


sum1 = tf.summary.scalar('loss', tf.reduce_sum(loss))
sum2 = tf.summary.scalar('mean_max_reward', tf.reduce_max(predicted_action))
merged_summary = tf.summary.merge([sum1, sum2]) #tf.summary.merge_all()


#optimizer = tf.train.AdamOptimizer().minimize(loss)
#optimizer = tf.train.RMSPropOptimizer(0.001).minimize(loss)
learning_rate_minimum = 0.00015
learning_rate = 0.0025
learning_rate_decay = 0.97
scale = 10000
learning_rate_decay_step = 5 * scale

learning_rate_step = tf.placeholder('int64', None, name='learning_rate_step')
learning_rate_op = tf.maximum(learning_rate_minimum,tf.train.exponential_decay(
      learning_rate,
      global_step,
      learning_rate_decay_step,
      learning_rate_decay,
      staircase=True))

optimizer = tf.train.RMSPropOptimizer(learning_rate_op, momentum=0.97, epsilon=0.001).minimize(loss, global_step=global_step)


mean_last_n_ph = tf.placeholder(tf.float32)
max_last_n_ph = tf.placeholder(tf.float32)
min_last_n_ph = tf.placeholder(tf.float32)
epsilon_ph = tf.placeholder(tf.float32)
sum3 = tf.summary.scalar("mean_last_n",mean_last_n_ph )
sum4 = tf.summary.scalar("max_last_n",max_last_n_ph )
sum5 = tf.summary.scalar("min_last_n",min_last_n_ph )
sum6 = tf.summary.scalar("epsilon",epsilon_ph )
sum7 = tf.summary.scalar("learning_rate",learning_rate_op )
merged_summary_meta = tf.summary.merge([sum3, sum4, sum5, sum6, sum7])

update_target_expr = []
for var, var_target in zip(sorted(q_func_vars, key=lambda v: v.name), sorted(target_q_func_vars, key=lambda v: v.name)):
    update_target_expr.append(var_target.assign(var))
update_target_expr = tf.group(*update_target_expr)
update_target = function([], [], updates=[update_target_expr])



In [ ]:
len(q_func_vars)

In [ ]:
sess = tf.InteractiveSession()
summary_writer = tf.summary.FileWriter('graphs/dqn/atari/'+str(logging_dir_n),sess.graph)
logging_dir_n +=1
sess.run(tf.global_variables_initializer())

In [ ]:
import cv2
def preprocess(observation):
    observation = cv2.cvtColor(cv2.resize(observation, (84, 110), interpolation = cv2.INTER_AREA), cv2.COLOR_BGR2GRAY)
    observation = observation[26:110,:]
    ret, observation = cv2.threshold(observation,1,255,cv2.THRESH_BINARY)
    return np.reshape(observation,(84,84,1))




In [ ]:
import time
import collections
replay_memory = [] # (state, action, reward, terminalstate, state_t+1)
epsilon = 1.0
decay_factor = 0.000001
MIN_EPSILON = 0.1
BATCH_SIZE = 32
MAX_LEN_REPLAY_MEMORY = 500000 # replay memory of one million most recent frames.
FRAMES_TO_PLAY = 50000000
MIN_FRAMES_FOR_LEARNING = 1000

observation = env.reset()
observation = preprocess(observation)
print(observation.shape)
plt.imshow(np.reshape(observation, (84,84)), cmap='gray')
plt.show()

total_reward_this_episode = 0
best_reward_so_far = 0
lastframes = collections.deque([observation]*4, 4)

def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)

last_rewards = list()
last_n = 100
mean_last_n = 0
min_last_n = 0
max_last_n = 0


for _ in range(4):
    action = env.action_space.sample() 
    newobservation, reward, terminal, info = env.step(action)
    lastframes.append(preprocess(newobservation))
observation = np.dstack(lastframes)    
for i_epoch in range(FRAMES_TO_PLAY):
    
    if i_epoch %10000 == 10:
        print("SETTING NETWORK PARAMETERS TO SAME" + str(i_epoch) + " " + str(epsilon))
        sess.run(update_target_expr)
        
    
    
    
    
    ### Select action
    if i_epoch%50==0:
        action = 1 # Press start once in a while
    elif random.random() < epsilon:
        action = env.action_space.sample() 
    else:
        pred_q = sess.run(predicted_action, feed_dict={networkstate:[observation]})
       # print(pred_q)

        action = np.argmax(pred_q)
    
    ### Anneal epsilon
    epsilon = max(epsilon-decay_factor, MIN_EPSILON)
    
    newobservation, reward, terminal, info = env.step(action)
    lastframes.appendleft(preprocess(newobservation))
    newobservation = np.dstack(lastframes)
   
#     plt.imshow(newobservation[:,:,:3])
#     plt.show()
    
    ### I prefer that my agent gets 0 reward if it dies
    if terminal: 
        reward = 0
    
    total_reward_this_episode += reward
        
    ### Add the observation to our replay memory
    replay_memory.append((observation, action, reward, terminal, newobservation))
    
    ### Reset the environment if the agent died
    if terminal:
        if total_reward_this_episode > best_reward_so_far:
            best_reward_so_far = total_reward_this_episode
            print("Reward this episode: " + str(total_reward_this_episode) + " epsilon: " + str(epsilon))
        last_rewards.append(total_reward_this_episode)
        
 
        mean_last_n = mean(last_rewards[-last_n:])
    
        min_last_n = min(last_rewards[-last_n:])
        max_last_n = max(last_rewards[-last_n:])
        total_reward_this_episode = 0
        newobservation = env.reset()
        lastframes.appendleft(preprocess(newobservation))
        newobservation = np.dstack(lastframes)
    observation = newobservation
    
    ### Learn once we have enough frames to start learning
    if len(replay_memory) > MIN_FRAMES_FOR_LEARNING: 
        experiences = random.sample(replay_memory, BATCH_SIZE)
     
        totrain = [] # (state, action, delayed_reward)
        
        ### Calculate the predicted reward

#         nextstates = [var[4] for var in experiences]
#         currentstates = [var[0] for var in experiences]
#         performedactions = [var[1] for var in experiences]
        
        
        ### Set the "ground truth": the value our network has to predict:
#         for index in range(BATCH_SIZE):
#             state, action, reward, terminalstate, newstate = experiences[index]
#             #print(terminalstate)
#             #totrain.append((state, action, delayedreward))
            
        ### Feed the train batch to the algorithm 
        states = [var[0] for var in experiences]
        actions = [var[1] for var in experiences]
        rewards = [var[2] for var in experiences]
        terminalstates = [float(var[3]) for var in experiences]
        nextstates = [var[4] for var in experiences]
        
      #  print(terminalstates)
      #  print(nextstates)
        
        _, l, summary = sess.run([optimizer, loss, merged_summary], 
                                 feed_dict={networkstate:states, 
                                            networkaction: actions, 
                                            networkreward: rewards, 
                                           nextnetworkstate: nextstates, 
                                           done_mask_ph: terminalstates})

   
        ### If our memory is too big: remove the first element
        if len(replay_memory) > MAX_LEN_REPLAY_MEMORY:
                replay_memory = replay_memory[1:]

        ### Show the progress 
        summary_writer.add_summary(summary, i_epoch)
        
        
        summary = sess.run(merged_summary_meta, feed_dict={mean_last_n_ph: mean_last_n, 
                                                           max_last_n_ph: max_last_n, 
                                                           min_last_n_ph: min_last_n,
                                                          epsilon_ph: epsilon})
#         summary = sess.run(merged_summary_meta, feed_dict={mean_last_n_ph: mean_last_n, 
#                                                           max_last_n_ph: float(max_last_n), 
#                                                           min_last_n: float(min_last_n),
#                                                           epsilon_ph: epsilon})
        summary_writer.add_summary(summary, i_epoch)
       
  

In [ ]:
for i_epoch in range(5000000, 50000000):
    
    if i_epoch %10000 == 10:
        print("SETTING NETWORK PARAMETERS TO SAME" + str(i_epoch) + " " + str(epsilon))
        sess.run(update_target_expr)
        
    
    
    
    
    ### Select action
    if i_epoch%50==0:
        action = 1 # Press start once in a while
    elif random.random() < epsilon:
        action = env.action_space.sample() 
    else:
        pred_q = sess.run(predicted_action, feed_dict={networkstate:[observation]})
       # print(pred_q)

        action = np.argmax(pred_q)
    
    ### Anneal epsilon
    epsilon = max(epsilon-decay_factor, MIN_EPSILON)
    
    newobservation, reward, terminal, info = env.step(action)
    lastframes.appendleft(preprocess(newobservation))
    newobservation = np.dstack(lastframes)
   
#     plt.imshow(newobservation[:,:,:3])
#     plt.show()
    
    ### I prefer that my agent gets 0 reward if it dies
    if terminal: 
        reward = 0
    
    total_reward_this_episode += reward
        
    ### Add the observation to our replay memory
    replay_memory.append((observation, action, reward, terminal, newobservation))
    
    ### Reset the environment if the agent died
    if terminal:
        if total_reward_this_episode > best_reward_so_far:
            best_reward_so_far = total_reward_this_episode
            print("Reward this episode: " + str(total_reward_this_episode) + " epsilon: " + str(epsilon))
        last_rewards.append(total_reward_this_episode)
        
 
        mean_last_n = mean(last_rewards[-last_n:])
    
        min_last_n = min(last_rewards[-last_n:])
        max_last_n = max(last_rewards[-last_n:])
        total_reward_this_episode = 0
        newobservation = env.reset()
        lastframes.appendleft(preprocess(newobservation))
        newobservation = np.dstack(lastframes)
    observation = newobservation
    
    ### Learn once we have enough frames to start learning
    if len(replay_memory) > MIN_FRAMES_FOR_LEARNING: 
        experiences = random.sample(replay_memory, BATCH_SIZE)
     
        totrain = [] # (state, action, delayed_reward)
        
        ### Calculate the predicted reward

#         nextstates = [var[4] for var in experiences]
#         currentstates = [var[0] for var in experiences]
#         performedactions = [var[1] for var in experiences]
        
        
        ### Set the "ground truth": the value our network has to predict:
#         for index in range(BATCH_SIZE):
#             state, action, reward, terminalstate, newstate = experiences[index]
#             #print(terminalstate)
#             #totrain.append((state, action, delayedreward))
            
        ### Feed the train batch to the algorithm 
        states = [var[0] for var in experiences]
        actions = [var[1] for var in experiences]
        rewards = [var[2] for var in experiences]
        terminalstates = [float(var[3]) for var in experiences]
        nextstates = [var[4] for var in experiences]
        
      #  print(terminalstates)
      #  print(nextstates)
        
        _, l, summary = sess.run([optimizer, loss, merged_summary], 
                                 feed_dict={networkstate:states, 
                                            networkaction: actions, 
                                            networkreward: rewards, 
                                           nextnetworkstate: nextstates, 
                                           done_mask_ph: terminalstates})

   
        ### If our memory is too big: remove the first element
        if len(replay_memory) > MAX_LEN_REPLAY_MEMORY:
                replay_memory = replay_memory[1:]

        ### Show the progress 
        summary_writer.add_summary(summary, i_epoch)
        
        
        summary = sess.run(merged_summary_meta, feed_dict={mean_last_n_ph: mean_last_n, 
                                                           max_last_n_ph: max_last_n, 
                                                           min_last_n_ph: min_last_n,
                                                          epsilon_ph: epsilon})
#         summary = sess.run(merged_summary_meta, feed_dict={mean_last_n_ph: mean_last_n, 
#                                                           max_last_n_ph: float(max_last_n), 
#                                                           min_last_n: float(min_last_n),
#                                                           epsilon_ph: epsilon})
        summary_writer.add_summary(summary, i_epoch)
       

In [ ]:
len(replay_memory)

In [ ]:
print(l)

In [ ]:
observation = env.reset()
term = False
predicted_q = []
frames = []

lastframes.appendleft(preprocess(observation))
newobservation = np.dstack(lastframes)
   
### Play till we are dead
for _ in range(300):
#while not term:
    rgb_observation = env.render(mode = 'rgb_array')
    frames.append(rgb_observation)
    
    
    lastframes.appendleft(preprocess(newobservation))
    newobservation = np.dstack(lastframes)
    print(newobservation.max())
        
    pred_q = sess.run(predicted_action, feed_dict={networkstate:[newobservation]})
    predicted_q.append(pred_q)
    
    if random.random() < 0.2:
        action = env.action_space.sample() 
    else:
        action = np.argmax(pred_q)
    
    observation, _, term, _ = env.step(action)
    if term: 
        env.reset()
    newobservation = observation
### Plot the replay!
print("Frames: " + str(len(frames)))
display_frames_as_gif(frames,filename_gif='dqn_run.gif')

In [ ]:
for _ in range(300):
    rgb_observation = env.render(mode = 'rgb_array')
    plt.imshow(rgb_observation)
    plt.show()
    
    action = int(input("input"))
    observation, _, term, _ = env.step(action)
    print(term)

In [ ]:
for var in target_q_func_vars:
    print(var.name)


In [ ]:
len(target_q_func_vars)

In [ ]:
trainable_only = False
tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES if trainable_only else tf.GraphKeys.GLOBAL_VARIABLES, scope=scope if isinstance(scope, str) else scope.name
    )

In [ ]:
tf.get_variable_scope().name




In [ ]:
"/" + relative_scope_name

In [ ]:
var_23 = [v for v in tf.global_variables() if v.name.startswith("actionnetwork")]
print(var_23)

In [ ]:
var_23 = [v for v in tf.global_variables() if v.name.startswith("valuenetwork")]
print(var_23)

In [ ]:
tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "actionnetwork")